# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample print(city_data_df.columns)
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,24.44,49,20,1.54,US,1689976033
1,1,ushuaia,-54.8000,-68.3000,-1.19,69,75,6.69,AR,1689975783
2,2,punta arenas,-53.1500,-70.9167,-2.16,92,100,2.47,CL,1689976632
3,3,greymouth,-42.4667,171.2000,5.22,75,14,3.63,NZ,1689976632
4,4,jatai,-17.8814,-51.7144,22.57,45,2,3.02,BR,1689976633


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.55,
    color="City"
)

# Display the map
city_data_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:

# Define the criteria for the ideal weather condition
# For example, let's assume you want to filter cities with a maximum temperature between 70 and 80 degrees Fahrenheit, and wind speed less than 10 mph.
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) &
    (city_data_df['Max Temp'] <= 31) &
    (city_data_df['Wind Speed'] <10 )
]

# Drop any rows with null values (if applicable)
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,24.44,49,20,1.54,US,1689976033
4,4,jatai,-17.8814,-51.7144,22.57,45,2,3.02,BR,1689976633
8,8,hilo,19.7297,-155.0900,27.84,74,100,3.09,US,1689976462
9,9,atafu village,-8.5421,-172.5159,28.79,70,64,4.83,TK,1689976041
10,10,nago,26.5881,127.9761,28.47,81,61,3.14,JP,1689976432


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,badger,US,64.8000,-147.5333,49,
1,ushuaia,AR,-54.8000,-68.3000,69,
2,punta arenas,CL,-53.1500,-70.9167,92,
3,greymouth,NZ,-42.4667,171.2000,75,
4,jatai,BR,-17.8814,-51.7144,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 kilometers)
params = {
    "type": "hotel",
    "radius": radius,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = {"type": "nearby", "lat": lat, "lon": lng}
    params["bias"] = {"lat": lat, "lon": lng}
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
badger - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
greymouth - nearest hotel: No hotel found
jatai - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
nago - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
dolisie - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
tromso - nearest hotel: No hotel found
remire-montjoly - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
magadan - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,badger,US,64.8000,-147.5333,49,No hotel found
1,ushuaia,AR,-54.8000,-68.3000,69,No hotel found
2,punta arenas,CL,-53.1500,-70.9167,92,No hotel found
3,greymouth,NZ,-42.4667,171.2000,75,No hotel found
4,jatai,BR,-17.8814,-51.7144,45,No hotel found
...,...,...,...,...,...,...
557,brookings,US,44.3114,-96.7984,65,No hotel found
558,lagoa,PT,39.0500,-27.9833,84,No hotel found
559,nguigmi,NE,14.2495,13.1092,53,No hotel found
560,boende,CD,-0.2167,20.8667,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot with hover information
city_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.75,
    color="City",
    hover_cols=["City", "Country", "Hotel Name", "Humidity"],
    title="City Humidity with Nearby Hotel Information"

)

# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)